<a href="https://colab.research.google.com/github/SugarC21/colab_Open_WebUI/blob/main/Colab_Open_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Open WebUI + Ollama in Colab**

This notebook sets up **Open WebUI** (Python 3.11) and **Ollama** on Google Colab. Both can optionally be installed in Google Drive to persist across sessions. **ngrok** is used to provide a public URL for Open WebUI (port 8081). Ollama runs in the background on port 11422.

### **Features**
1. **Install** Open WebUI in a Python 3.11 virtual environment.
2. **Optionally** persist both Open WebUI and Ollama in Google Drive.
3. **Expose** Open WebUI via ngrok.
4. **Hide** Ollama from the public URL (internal only).


In [ ]:
#@title **Setup**
use_gdrive = True #@param {type:"boolean"}
use_ngrok_auth = False #@param {type:"boolean"}

import os

# If Google Drive is enabled, mount and set BASE_PATH to persist.
BASE_PATH = "/content"
if use_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/Open-WebUI"
    os.makedirs(BASE_PATH, exist_ok=True)

print("Using base path:", BASE_PATH)
print("Using ngrok auth token:", use_ngrok_auth)

## (Optional) Using Authentication Token
1. Toggle `use_ngrok_auth` above.
2. Provide your token as `ngrok_auth_token` through either Colab secrets or an environment variable.
   - **Colab secrets** example:
     ```python
     from google.colab import userdata
     token = userdata.get('ngrok_auth_token')
     ```
   - **Environment variable** example:
     ```bash
     %env ngrok_auth_token=YOUR_TOKEN
     ```

## Install Dependencies
Installs Python 3.11, venv, dev packages, and system tools.

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y python3.11 python3.11-venv python3.11-dev pciutils lshw

## Install Ollama
We always use the **official script**:
```
!curl -fsSL https://ollama.com/install.sh | sh
```

- If `use_gdrive` is **True**, we **copy** the installed Ollama binary into Drive for persistence.
- Otherwise, Ollama remains ephemeral.

In [ ]:
ollama_drive_path = os.path.join(BASE_PATH, 'ollama')
ollama_bin_drive = os.path.join(ollama_drive_path, 'ollama')

if use_gdrive:
    # If we have a persisted binary, skip reinstallation
    if os.path.exists(ollama_bin_drive):
        print("Ollama binary already present in Drive.")
        print("(To reinstall/update, remove or overwrite the existing file in Drive.)")
    else:
        print("Installing Ollama system-wide (one time)...")
        !curl -fsSL https://ollama.com/install.sh | sh

        # Attempt to locate the installed Ollama binary (should be /usr/local/bin/ollama)
        import subprocess
        which_ollama = subprocess.check_output(['which', 'ollama']).decode('utf-8').strip()

        print(f"Copying Ollama from {which_ollama} to Drive...")
        os.makedirs(ollama_drive_path, exist_ok=True)
        !cp "$which_ollama" "$ollama_bin_drive"
        !chmod +x "$ollama_bin_drive"
        print("Ollama persisted in Drive at:", ollama_bin_drive)
else:
    print("Installing Ollama system-wide (ephemeral)")
    !curl -fsSL https://ollama.com/install.sh | sh
    print("Ollama installed for this session.")

## Set Up Virtual Environment & Install Open WebUI

In [ ]:
import os

venv_path = os.path.join(BASE_PATH, "venv")
if not os.path.exists(venv_path):
    print("Creating Python 3.11 virtual environment...")
    !python3.11 -m venv "$venv_path"

print("Upgrading pip...")
!"$venv_path/bin/python" -m pip install --upgrade pip

print("Installing Open WebUI...")
!"$venv_path/bin/pip" install open-webui

print("Open WebUI installation complete.")

## Create a Script to Start Both Servers
Ollama runs on **port 11422**. Open WebUI runs on **port 8081**.

If using Drive, we run Ollama from the Drive copy. Otherwise, we run the system-wide install.

In [ ]:
import os

server_script_path = os.path.join(BASE_PATH, 'start_servers.py')

if use_gdrive:
    ollama_bin = ollama_bin_drive  # Use the persisted copy
else:
    ollama_bin = 'ollama'          # Use system-wide ephemeral

script_content = f'''\
import subprocess, threading, time\n\n
OLLAMA_CMD = "{ollama_bin}"\n\n
def start_ollama():\n
    subprocess.run([OLLAMA_CMD, 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)\n\n
def start_open_webui():\n
    subprocess.run(['./venv/bin/open-webui', 'serve', '--port', '8081'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)\n\n
threading.Thread(target=start_ollama).start()\n
time.sleep(5)\n
threading.Thread(target=start_open_webui).start()\n
'''

with open(server_script_path, 'w') as f:
    f.write(script_content)

print("Created:", server_script_path)

## Start Servers & Expose Open WebUI via ngrok
Open WebUI is on **port 8081**, Ollama on **port 11422** (not publicly exposed).

In [ ]:
!pip install pyngrok --quiet

import time, os
from pyngrok import ngrok
from google.colab import userdata

def get_ngrok_token():
    # 1) Check Colab secrets
    secret_token = userdata.get('ngrok_auth_token')
    if secret_token:
        return secret_token
    # 2) Check environment variable
    env_token = os.environ.get('ngrok_auth_token', '')
    if env_token:
        return env_token
    return ''

if use_ngrok_auth:
    token = get_ngrok_token()
    if token:
        ngrok.set_auth_token(token)
        print("ngrok auth token set.")
    else:
        print("No 'ngrok_auth_token' found. Proceeding without auth token.")

print("Starting servers...")
venv_path = os.path.join(BASE_PATH, 'venv')
server_script_path = os.path.join(BASE_PATH, 'start_servers.py')
!"$venv_path/bin/python" "$server_script_path" &
time.sleep(20)

webui_tunnel = ngrok.connect(8081, "http")
print("Open WebUI URL:", webui_tunnel.public_url)

# Hidden tunnel for Ollama
ollama_tunnel = ngrok.connect(11422, "http")

print("\nSetup complete.")